<a href="https://colab.research.google.com/github/abhijitpal1247/animatediff_exp/blob/main/animatediff_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install peft
pip install --upgrade transformers
pip install diffusers
pip install python-dotenv
pip install openai==0.28
mkdir cola_weights
wget https://civitai.com/api/download/models/267807 -O cola_weights/pytorch_lora.safetensors

In [ ]:
# @title Enter product details { display-mode: "form" }

from dotenv import load_dotenv

load_dotenv()

company_name = 'coca-cola' # @param {type:"string"}
product = 'cola-bottle' # @param {type:"string"}

In [ ]:
import os
import time
import random
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

# define a retry decorator
def retry_with_exponential_backoff(
    func,
    initial_delay: float = 2,
    exponential_base: float = 2,
    jitter: bool = True,
    max_retries: int = 30,
    errors: tuple = (openai.error.RateLimitError, openai.error.APIError, openai.error.Timeout, openai.error.ServiceUnavailableError),
):
    """Retry a function with exponential backoff."""

    def wrapper(*args, **kwargs):
        # Initialize variables
        num_retries = 0
        delay = initial_delay

        # Loop until a successful response or max_retries is hit or an exception is raised
        while True:
            try:
                return func(*args, **kwargs)

            # Retry on specific errors
            except errors as e:
                print(str(e))
                # Increment retries
                num_retries += 1

                # Check if max retries has been reached
                if num_retries > max_retries:
                    raise Exception(
                        f"Maximum number of retries ({max_retries}) exceeded."
                    )

                # Increment the delay
                delay *= exponential_base * (1 + jitter * random.random())

                # Sleep for the delay
                time.sleep(delay)

            # Raise exceptions for any errors not specified
            except Exception as e:
                raise e

    return wrapper

@retry_with_exponential_backoff
def completions_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

In [ ]:
def get_openai_completion(model, max_tokens, temperature, n, messages,
                          return_completions_only=False):
    response= completions_with_backoff(model=model, max_tokens=max_tokens,
                                       temperature=temperature, n=n,
                                       messages=messages)

    if return_completions_only:
        continuations=[]
        for res in response["choices"]:
            if "message" in res:
                continuations.append(res["message"]["content"])
            else:
                continuations.append(res["text"])
        return continuations
    return response

In [ ]:
system_message = "You are a storyboard writer. You write the story board descriptions for ads "
prompt = f"""provide a storyboard for ad for the product {product} of this company: {company_name}
The ad will be a gif based video so our timeframe is very small.
Instructions:
1. Be as descriptive as possible
2. Describe the background, the relative-product placement and every other minute detail
3. Describe the entire set of actions that takes place in the ad video
4. Keep the storyboard concise, only one line and is of max 40 words describing the entire ad
5. Do not provide frame wise or scene wise description, provide overall idea or concept of ad
6. It should just have small set of actions, so that it fits within the timeframe
9. You do not have any type of overlay available so you can not display logo
10. You cannot display any tagline or any text
11. There is no audio associated with the video
12. You have a camera which can take a fixed aspect ratio video

Storyboard:"""

In [ ]:
messages = [{"role": "system", "content": system_message}]
messages.append({"role": "user", "content": prompt})
completions = get_openai_completion(model="gpt-3.5-turbo",
                                    max_tokens=1000, n=1,
                                    temperature=1.0, messages=messages,
                                    return_completions_only=True)
script = completions[0]

In [ ]:
script

'A person takes a refreshing sip of Coca-Cola from a cola bottle, their face lighting up with delight as they enjoy the fizzy drink.'

In [ ]:
system_message = "You are a prompt writter for text to video model. Given description you will write a prompt that will generate great video"
prompt = f"""provide the prompt for the following video description: {script}
Instructions:
The focus of the prompt should be on : {product} of {company_name} company
Like shown in the example, describe the {product} and the relative action that is taking place in the video
keep the background description simple
keep the prompt size small like 30-40 word length
Follow the below examples and be creative
Mention the company name along with the product

Example:
1. (a person drinking from a bottle of coca cola, sitting on top of ice covered ground and water splashing with a red background and a red background), red background, simple background, english text,Best quality,masterpiece,ultra high res,
2. (a bottle of coca cola is being thrown into the air with ice cubes on the bottom of it), no humans, simple background, white background, still life, english text, coca-cola, bottle, soda, food focus, realistic,Best quality,masterpiece,ultra high res
3. (a coca cola bottle with a red circle around it and daisies in the background with a blue sky and white daisies), no humans, flower, white flower, still life, bottle, water, english text,
4. (a person holding coca cola can) intricate playground atmosphere, atmospheric lighting, 4k UHD, hyper detailed, vibrant colors playground background, epic composition, octane render, sharp focus, high resolution isometric, in the center shot,
5. (a big bottle in the playground, inside the bottle there's a coca cola can) intricate playground atmosphere, atmospheric oliva lighting, on the table, 4k UHD, hyper detailed, vibrant colors playground background, epic composition, octane render, sharp focus, high resolution isometric, in the center shot
6. (a poster of a coca cola advertisement with six bottles of coke on it and a bag of soda on the side),no humans,english text,still life,artist name,border,Best quality,masterpiece,ultra high res,
7. (a rabbit sitting next to a coca cola bottle and jumping in a field of grass and flowers with a sun shining behind it), no humans, rabbit, coca-cola, animal focus, animal, plant, outdoors, signature, bottle, nature, grass, tree,Best quality,masterpiece,ultra high res
8. (A person taking a Coca-Cola bottle from a fridge), beautiful interior, high quality, 8k, warm light
9. (A rabbit holding a coca cola bottle in its paws and sipping it), in a field of grass and weeds with the sun shining behind it,no humans,bottle,blurry,animal,animal focus,rabbit,outdoors,grass,blurry background,coca-cola,plant,realistic,mushroom,whiskers, Best quality,masterpiece,ultra high res

prompt:"""
messages = [{"role": "system", "content": system_message}]
messages.append({"role": "user", "content": prompt})
completions = get_openai_completion(model="gpt-3.5-turbo",
                                  max_tokens=1000, n=1,
                                  temperature=1.0, messages=messages,
                                  return_completions_only=True)
prompt = completions[0]

In [ ]:
prompt

'(a person takes a refreshing sip of Coca-Cola from a classic glass bottle, their face lighting up with delight as they enjoy the fizzy drink), simple background, product focus, english text, beverage, enjoyment, high quality, Best quality, ultra high res.'

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
from IPython.display import display

text_input = widgets.Text(
    value=prompt,
    description='Edit prompt',
    disabled=False,
    layout = widgets.Layout(width='2000px')
)

display(text_input)

Text(value='kekokelev2, (th3r0ck takes a refreshing sip of Coca-Cola from a classic glass bottle, muscular, se…

In [ ]:
import torch
from diffusers import AnimateDiffPipeline, LCMScheduler, MotionAdapter
from diffusers.utils import export_to_gif

adapter = MotionAdapter.from_pretrained("wangfuyun/AnimateLCM")
pipe = AnimateDiffPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", motion_adapter=adapter, variant="fp16")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config, beta_schedule="linear")

pipe.load_lora_weights("cola_weights/", adapter_name="cola-lora")
pipe.load_lora_weights("wangfuyun/AnimateLCM", weight_name="AnimateLCM_sd15_t2v_lora.safetensors", adapter_name="lcm-lora")
pipe.load_lora_weights("guoyww/animatediff-motion-lora-tilt-up", adapter_name="tilt-up")
# pipe.load_lora_weights(
#     "guoyww/animatediff-motion-lora-zoom-out", adapter_name="zoom-out",
# )
# pipe.load_lora_weights(
#     "guoyww/animatediff-motion-lora-pan-left", adapter_name="pan-left",
# )

pipe.set_adapters(["lcm-lora", "tilt-up", "cola-lora"],
 [1.0, 0.8, 0.8])
pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

The config attributes {'center_input_sample': False, 'flip_sin_to_cos': True, 'freq_shift': 0, 'attention_head_dim': 8} were passed to UNetMotionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


In [ ]:
output = pipe(
    prompt=text_input.value,
    negative_prompt="bad quality, worse quality, low resolution, ng_deepnegative_v1_75t,(worst quality:2),(low quality:2),(normal quality:2),lowres,bad anatomy,bad hands,normal quality,((monochrome)),((grayscale)),watermark,(negative_hand-neg:0.6),(NegfeetV2:0.5),(EasyNegativeV2:0.6),extra fingers,deformed hands,polydactyl:1.3",
    num_frames=16,
    guidance_scale=1.5,
    num_inference_steps=25,
    generator=torch.Generator("cpu").manual_seed(0),
)
frames = output.frames[0]
export_to_gif(frames, "animatelcm-motion-lora.gif")

  0%|          | 0/25 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


'animatelcm-motion-lora.gif'